In [1]:
import cv2
import os
import os.path as osp
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
from skimage.morphology import skeletonize
from skimage.util import invert
from nebula.common import to_scale_one, write_pickle, read_pickle, to_pca
from nebula.data.yg_ar.setup_data_image_hard import split_ar_anim_df
from sklearn.decomposition import PCA

c:\users\aphri\miniconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] The specified procedure could not be found
  warn(f"Failed to load image Python extension: {e}")


In [2]:
def display(frame):

    # display results
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(4, 4))

    ax.imshow(frame, cmap=plt.cm.gray)
    ax.axis('off')

    fig.tight_layout()
    plt.show()
    
    
def display2(f1, f2):
    
    # display results
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 4),
                             sharex=True, sharey=True)

    ax = axes.ravel()

    ax[0].imshow(f1, cmap=plt.cm.gray)
    ax[0].axis('off')
    ax[0].set_title('image 1', fontsize=20)

    ax[1].imshow(f2, cmap=plt.cm.gray)
    ax[1].axis('off')
    ax[1].set_title('image 2', fontsize=20)

    fig.tight_layout()
    plt.show()
    

def get_loc_frame_from_data(data, loc=0.5):
    count = data.get(cv2.CAP_PROP_FRAME_COUNT)
    f_loc = int((count-1) * loc)
    
    data.set(cv2.CAP_PROP_POS_FRAMES, f_loc)
    ret, frame = data.read()
    
    return frame
    
    
def get_loc_frame(path, loc=0.5):
    data = cv2.VideoCapture(path)
    return get_loc_frame_from_data(data, loc=loc)


def get_n_frames(path, n=8):
    data = cv2.VideoCapture(path)
    res = [get_loc_frame_from_data(data, loc=i/n) for i in range(n)]
    
    return res


def get_locs_frames(path, locs=[0.15, 0.5, 0.85]):
    data = cv2.VideoCapture(path)
    res = [get_loc_frame_from_data(data, loc=loc) for loc in locs]
    return res
    

def to_gray_one(frame):
    return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


def to_gray(frames):
    res = [to_gray_one(f) for f in frames]
    return res


def crop_one(frame, top, bottom, left, right):
    return frame[top:-bottom, left:-right]


def crop(frames, top, bottom, left, right):
    res = [crop_one(f, top, bottom, left, right) for f in frames]
    return res


def save_frame(path, frame):
    cv2.imwrite(path, frame)
    

def read_gray_frame(path):
    return cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    
    
def to_jpg_name(file):
    return file.split(".")[0] + ".jpg"


def to_pickle_name(file):
    return file.split(".")[0] + ".pkl"


def get_files(dir, format=None):
    res = []
    for filename in os.listdir(dir):
        if format is None or filename.split(".")[-1] == format:
            res.append(filename)
    return res


def filename_to_labels(filename):
    
    filename = filename.split(".")[0]
    words = filename.split("_")
    
    orientation = words[-1]
    xangle = words[-2]
    yangle = words[-3]
    pants = words[-4]
    cloth = words[-6]
    hair = words[-8]
    action_type = words[-10]
    label = "_".join(words[:-10])
    
    return (
        orientation, 
        xangle, 
        yangle, 
        pants, 
        cloth, 
        hair, 
        action_type, 
        label
    )


def filename_to_labels_hard(filename):
    
    filename = filename.split(".")[0]
    words = filename.split("_")
    
    action = "_".join(words[:-12])
    action_type = words[-12]
    
    label_a = action
    label_at = "_".join([action, action_type])
    
    return (
        label_a,
        label_at,
        filename
    )


def read_pickle(dir):
    with open(dir, 'rb') as handle:
        b = pickle.load(handle)
    return b


def write_pickle(dir, data):
    with open(dir, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
def convert_middle_frame(
    base_dir,
    save_dir,
    overwrite=False,
    crop=dict(top=70, bottom=30, left=50, right=50)
):
    if not osp.exists(save_dir):
        os.makedirs(save_dir)
    
    files = get_files(base_dir)
    count = len(files)
    prev_progress = 0
    for i, file in enumerate(files):
        save_file = osp.join(save_dir, to_jpg_name(file))
        if osp.exists(save_file) and not overwrite:
            continue
        
        path = osp.join(base_dir, file)
        frame = to_gray_one(crop_one(get_loc_frame(path), **crop))
        
        save_frame(save_file, frame)
        
        cur_progress = int((i+1)*100/count)
        if cur_progress >= prev_progress + 2:
            print(f"progress: {cur_progress}%")
            prev_progress = cur_progress
            

def df_middle_frame(
    image_dir,
    save_path,
):
    files = get_files(image_dir)
    count = len(files)
    print(count)
    prev_progress = 0
    res = []
    for i, f in enumerate(files):
        frame = read_gray_frame(osp.join(image_dir, f))
        (
            label_a,
            label_ac,
            fname
        ) = filename_to_labels_hard(f)
        
        res.append(
            (
                frame, 
                label_a,
                label_ac,
                fname
            )
        )
        
        cur_progress = int((i+1)*100/count)
        if cur_progress >= prev_progress + 2:
            print(f"progress: {cur_progress}%")
            prev_progress = cur_progress

    df = pd.DataFrame(data=dict(zip(
        [
            "image", 
            "label_a",
            "label_at",
            "file_name"
        ], 
        np.transpose(res)
    )))
    
    df.to_pickle(save_path)
    
    return df

In [4]:
def convert_locs_frames(
    base_dir,
    save_dir,
    overwrite=False,
    crop=dict(top=70, bottom=30, left=50, right=50),
    locs=[0.15, 0.5, 0.85]
):
    if not osp.exists(save_dir):
        os.makedirs(save_dir)
    
    files = get_files(base_dir)
    count = len(files)
    prev_progress = 0
    for i, file in enumerate(files):
        save_file = osp.join(save_dir, to_pickle_name(file))
        if osp.exists(save_file) and not overwrite:
            continue
        
        path = osp.join(base_dir, file)
        frames = get_locs_frames(path, locs=locs)
        frames = [to_gray_one(crop_one(f, **crop)) for f in frames]
        
        write_pickle(save_file, frames)
        
        cur_progress = int((i+1)*100/count)
        if cur_progress >= prev_progress + 2:
            print(f"progress: {cur_progress}%")
            prev_progress = cur_progress
            

def df_locs_frames(
    pickle_dir,
    save_path,
):
    files = get_files(pickle_dir)
    count = len(files)
    print(count)
    prev_progress = 0
    res = []
    for i, f in enumerate(files):
        frames = read_pickle(osp.join(pickle_dir, f))
        (
            label_a,
            label_ac,
            fname
        ) = filename_to_labels_hard(f)
        
        res.append(
            (
                frames, 
                label_a,
                label_ac,
                fname
            )
        )
        
        cur_progress = int((i+1)*100/count)
        if cur_progress >= prev_progress + 2:
            print(f"progress: {cur_progress}%")
            prev_progress = cur_progress

    df = pd.DataFrame(data=dict(zip(
        [
            "images", 
            "label_a",
            "label_at",
            "file_name"
        ], 
        np.transpose(res)
    )))
    
    df.to_pickle(save_path)
    
    return df

In [5]:
from PIL import Image
import cv2 as cv
from scipy import ndimage
import random


def zoom(frame, x, y, z=100):
    img = Image.fromarray(frame)
    w, h = img.size
    z2 = z * 2/100.0
    img = img.crop((x - w / z2, y - h / z2, 
                    x + w / z2, y + h / z2))
    img = img.resize((w, h), Image.LANCZOS)
    return np.asarray(img)


def rotate(frame, degree):
    return ndimage.rotate(frame, degree, reshape=False)


def random_zoom(frame, min_loc=-10, max_loc=10, min_z=100, max_z=130):
    
    h, w = frame.shape
    
    x_offset = random.randint(min_loc, max_loc)
    y_offset = random.randint(min_loc, max_loc)
    z = random.randint(min_z, max_z)
    
    return zoom(frame, int(w/2)+x_offset, int(h/2)+y_offset, z)


def random_rotate(frame, min_d=-10, max_d=10):
    
    deg = random.randint(min_d, max_d)
    return rotate(frame, deg)
    
    

In [6]:
base_dir = 'C:/Users/aphri/Documents/t0002/pycharm/data/yg_ar/videos_medium'
save_dir = 'C:/Users/aphri/Documents/t0002/pycharm/data/yg_ar/middle_image_medium'
image_pickle_path = 'C:/Users/aphri/Documents/t0002/pycharm/data/yg_ar/image_medium_df.pkl'

In [7]:
df2 = read_pickle(image_pickle_path)

In [8]:
def point2_scale(df):
    df = df.copy()
    df["image"] = df["image"].apply(lambda x: to_scale_one(x, scale=0.2).flatten())
    df["image"] = df["image"] / 225
    random_seed = 1
    train_df, test_df, valid_df = split_ar_anim_df(df, random_seed)
    train_df = train_df.sample(frac=1, random_state=random_seed)  # randomize train_df
    return df, train_df, test_df, valid_df

In [9]:
df, train_df, test_df, valid_df = point2_scale(df2)

In [10]:
data_path = 'C:/Users/aphri/Documents/t0002/pycharm/data/yg_ar/image_medium_ds_point2_scale.pkl'
write_pickle(data_path, (df, train_df, test_df, valid_df))

In [8]:
df2["image"] = df2["image"].apply(lambda x: to_scale_one(x, scale=0.5))

In [9]:
weights, eigen_imgs, pca = to_pca(df2, n_components=256)

In [19]:
df = df2.copy()
df["image"] = pd.Series(data=[w for w in weights])

In [21]:
random_seed = 1
train_df, test_df, valid_df = split_ar_anim_df(df, random_seed)
train_df = train_df.sample(frac=1, random_state=random_seed)  # randomize train_df

In [22]:
data_path = 'C:/Users/aphri/Documents/t0002/pycharm/data/yg_ar/image_medium_ds_pca.pkl'
write_pickle(data_path, (df, train_df, test_df, valid_df))